# Evaluating the mutation functio

This time, we're going to use a slightly different market setup (to avoid the pmax < pmin error).

Instead of using market shocks, we'll use an offset function to dynamically alter the market. 

In [1]:
import BSE
import numpy as np
import csv
import matplotlib.pyplot as plt
import math
import importlib
importlib.reload(BSE)

# Use this to plot trades of a single experiment
def plot_trades(trial_id):
    prices_filename = trial_id + '_transactions.csv'
    x = np.empty(0)
    y = np.empty(0)
    with open(prices_filename, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            time = float(row[1])
            price = float(row[2])
            x = np.append(x,time)
            y = np.append(y,price)

    plt.plot(x, y, 'x') 
    
# Use this to run an experiment n times and plot all trades
def n_runs_plot(n, trial_id, start_time, end_time, traders_spec, order_sched, dump_all, verbose):
    x = np.empty(0)
    y = np.empty(0)

    for i in range(n):
        trialId = trial_id + '_' + str(i)
        tdump = open(trialId + '_avg_balance.csv','w')

        BSE.market_session(trialId, start_time, end_time, traders_spec, order_sched, tdump, dump_all, verbose)
        
        tdump.close()
        
        if (dump_all):
            with open(trialId + '_transactions.csv', newline='') as csvfile:
                reader = csv.reader(csvfile)
                for row in reader:
                    time = float(row[1])
                    price = float(row[2])
                    x = np.append(x,time)
                    y = np.append(y,price)

    plt.plot(x, y, 'x');
    
# Use this to run an experiment n times and plot all trades, from a specified index onwards
def n_runs_plot_with_index(n, trial_id, index, start_time, end_time, traders_spec, order_sched, dump_all):
    x = np.empty(0)
    y = np.empty(0)

    for i in range(index, n):
        trialId = trial_id + '_' + str(i)
        tdump = open(trialId + '_avg_balance.csv','w')

        BSE.market_session(trialId, start_time, end_time, traders_spec, order_sched, tdump, dump_all, False)
        
        tdump.close()
        
        if (dump_all):
            with open(trialId + '_transactions.csv', newline='') as csvfile:
                reader = csv.reader(csvfile)
                for row in reader:
                    time = float(row[1])
                    price = float(row[2])
                    x = np.append(x,time)
                    y = np.append(y,price)

    plt.plot(x, y, 'x', color='black');
    
def create_trader_spec(buyers_spec, sellers_spec):
    traders_spec = {
        "sellers": sellers_spec,
        "buyers": buyers_spec
    }
    return traders_spec

def create_supply_demand_schedule(start_time, end_time, ranges, stepmode):
    schedule = {
        "from": start_time,
        "to": end_time,
        "ranges": ranges,
        "stepmode": stepmode
    }
    return schedule

def create_order_schedule(supply_schedule,demand_schedule,interval,timemode):
    order_sched = {
        "sup": supply_schedule,
        "dem": demand_schedule,
        "interval": interval,
        "timemode": timemode
    }
    return order_sched

In [2]:
# Define market conditions for experiment, and generate 100 CSV files with the results 
# Note: please change the value of k in BSE.py and reimport before running this function
def run_for_k(k):

    trial_id = str(k)
    n_trials = 100
    dump_all = False
    verbose = False

    eval_time = 300
    adaptive_steps = 10
    total_time = k * eval_time * (adaptive_steps + 1) # allow time to profit from 10th adaptive step
    
    start_time = 0
    end_time = total_time

    # Set length of trading day
    order_interval = 10 

    buyers_spec = [("ZIP",19),("PRSH",1)]
    sellers_spec = buyers_spec
    traders_spec = create_trader_spec(buyers_spec, sellers_spec)
    
    # Introduce market shocks
    initial_range = [(300, 350)]
    # Small upwards shock
    shock_one_time = end_time / 3
    shock_one_range = [(360, 410)]
    # Large upwards shock
    shock_two_time = end_time / 2
    shock_two_range = [(510, 560)]

    # Define the supply schedule (how limit prices are distributed to sellers, from a range)
    supply_schedule = [
        create_supply_demand_schedule(start_time, shock_one_time, initial_range, "fixed"),
        create_supply_demand_schedule(shock_one_time, shock_two_time, shock_one_range, "fixed"),
        create_supply_demand_schedule(shock_two_time, end_time, shock_two_range, "fixed")
    ]
    
    demand_schedule = supply_schedule

    # Define the demand schedule (how limit prices are distributed to buyers, from a range)
    order_sched = create_order_schedule(supply_schedule, demand_schedule, order_interval, "drip-poisson")

    n_runs_plot(n_trials, trial_id, start_time, end_time, traders_spec, order_sched, dump_all, verbose)


In [ ]:
# Run the experiment with your choice of k
run_for_k(2)